In [1]:
import math
import torch
from torch import nn
from torch import Tensor
from torch.nn  import functional as F
import gpytorch
from matplotlib import pyplot as plt
import sys
from decimal import Decimal
sys.path.append("..")
import vvkernels as vvk
import vvk_rbfkernel as vvk_rbf
import vvmeans as vvm
import vvlikelihood as vvll
from vfield import VField
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
sample_x = torch.linspace(0, 1, 300)
k = 2
N = sample_x.shape[0]
indices = torch.randperm(N)[:k]
train_x = sample_x[indices]
#train_x = train_x.reshape(train_x.shape[0],1)

train_y = torch.stack([
    torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,
    torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,
], -1)

def g_theta(theta):
    return torch.linspace(0, 1, 10)

In [3]:

x_train = train_x #loc #torch.linspace(0, 1, 10)
y_train = train_y #v  #torch.stack([torch.sin(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,torch.cos(train_x * (2 * math.pi)) + torch.randn(train_x.size()) * 0.2,], -1)

class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        a = torch.ones(2,2)
        chol_q = torch.tril(a)
        self.mean_module = vvm.TensorProductSubMean(gpytorch.means.ConstantMean(), num_tasks = 2)
#         self.covar_module = gpytorch.kernels.MultitaskKernel(
#             gpytorch.kernels.RBFKernel(), num_tasks=2, rank=1
#         )
        self.covar_module = vvk.TensorProductKernel(vvk_rbf.vvkRBFKernel(), a[0,0], a[1,0], a[1,1], num_tasks = 2, rank =1,  task_covar_prior=None)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x,x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)
    
    


In [4]:
###hyperparameters optimization###
def hyper_opti(g_theta1, agg_data, training_iter):
    likelihood = vvll.TensorProductLikelihood(num_tasks = 2)
    model = MultitaskGPModel(g_theta1, agg_data.reshape(g_theta1.shape[0],2), likelihood)
    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(),  lr=0.1)  # Includes GaussianLikelihood parameters

    for i in range(training_iter):
        optimizer.zero_grad()
        output = model(g_theta1)
        output_ll = likelihood(output)

        loss = -likelihood.get_mll(agg_data,output_ll)
        loss.backward(retain_graph=True)

        print('Iter %d/%d - Loss hyperparam: %.3f' % (i + 1, training_iter, loss.item()))
        optimizer.step()
    return model, likelihood


In [5]:
class param_opti(nn.Module):
    def __init__(self, _theta2,k):
        super(param_opti, self).__init__()
        N = _theta2.shape[0]
        indices = torch.randperm(N)[:k]
        sampled_values= _theta2[indices]
        sampled_values = sampled_values.reshape(sampled_values.shape[0], 1)
        self.g_theta2 = nn.Parameter(sampled_values)
        
    def forward(self):
        
        return (self.g_theta2)

In [6]:
def conduct_param_opti(x,g_theta1, agg_data, model, likelihood, training_iter):
    m = x.shape[0]
    f_target = 0.5 *torch.sqrt(Tensor([2.])) * torch.ones(2 * m, 1)  #torch.zeros(2,1)
#     f_target[0] = 0.5 *torch.sqrt(Tensor([2.]))
#     f_target[1] = 0.5 * torch.sqrt(Tensor([2.]))
    #f_target = f_target.reshape(2,1)

    
    _par = param_opti(sample_x,1)

    optimizer = torch.optim.Adam(_par.parameters(), lr=0.1)
    
    for i in range(training_iter):
        optimizer.zero_grad()
        g_theta2 = _par.forward()
        
        loss1 = -likelihood.get_ell(agg_data,f_target,x, g_theta1, g_theta2, model.mean_module, model.covar_module)
        loss1.backward(retain_graph=True)
        print('Iter %d/%d - Loss theta2: %.3f' % (i + 1, training_iter, loss1.item()))
        optimizer.step()
    return loss1, g_theta2

In [7]:
class design_opti(nn.Module):
    def __init__(self, _x):
        super(design_opti, self).__init__()
        self.x_design = nn.Parameter(_x)
        
    def forward(self):
        return self.x_design

In [8]:
def conduct_design_opti(x0, g_theta1, agg_data, model, likelihood, training_iter_out, training_iter_in):
    design = design_opti(x0)
    optimizer = torch.optim.Adam(design.parameters(), lr=0.1)
    with torch.autograd.set_detect_anomaly(True):
        for i in range(training_iter_out):
            optimizer.zero_grad()
            x_d = design.forward()
            loss2, g_theta2 = conduct_param_opti(x_d, g_theta1, agg_data, model, likelihood,training_iter_in)
            loss2.backward(retain_graph=True)
            print('Iter %d/%d - Loss design: %.3f' % (i + 1, training_iter_out, loss2.item()))
            optimizer.step()
    return x_d, g_theta2
    
    

In [11]:
iter_hp = 50
iter_design = 20
iter_param = 30
g_theta1 = []
agg_data = []
g_theta1.append((x_train).flatten())
g_theta11 = torch.cat(g_theta1)
g_theta11= g_theta11.unsqueeze(-1)

agg_data.append(y_train.flatten())
agg_data1 = torch.cat(agg_data)

x0 = torch.linspace(0, 1, 1)
x0 = x0.reshape(x0.shape[0],1)

for i in range(30):
    print('START HYPERPARAMETERS optimization')
    model, likelihood = hyper_opti(g_theta11,agg_data1,iter_hp)
    print('END HYPERPARAMETERS optimization')
    x0_new, g_theta2 = conduct_design_opti(x0, g_theta11,agg_data1, model, likelihood, iter_design, iter_param)
    x0 = x0_new
    model.eval()
    likelihood.eval()
    pred = likelihood(model(g_theta2))
    y_train_new = (pred.mean)
    agg_data.append(y_train_new.flatten())
    agg_data1 = torch.cat(agg_data)
    
    g_theta1.append(g_theta2.flatten())
    g_theta11 = torch.cat(g_theta1)
    g_theta11= g_theta11.unsqueeze(-1)

print(x0_new)
    

START HYPERPARAMETERS optimization
kjhgfd
tensor([[1., 1.],
        [1., 2.]], grad_fn=<MmBackward>)
lkjhgfd
Iter 1/50 - Loss hyperparam: 5.927
kjhgfd
tensor([[0.8100, 0.8100],
        [0.8100, 1.6200]], grad_fn=<MmBackward>)
lkjhgfd
Iter 2/50 - Loss hyperparam: 5.601
kjhgfd
tensor([[0.6398, 0.6399],
        [0.6399, 1.2799]], grad_fn=<MmBackward>)
lkjhgfd
Iter 3/50 - Loss hyperparam: 5.258
kjhgfd
tensor([[0.4894, 0.4896],
        [0.4896, 0.9793]], grad_fn=<MmBackward>)
lkjhgfd
Iter 4/50 - Loss hyperparam: 4.895
kjhgfd
tensor([[0.3587, 0.3590],
        [0.3590, 0.7182]], grad_fn=<MmBackward>)
lkjhgfd
Iter 5/50 - Loss hyperparam: 4.512
kjhgfd
tensor([[0.2478, 0.2482],
        [0.2482, 0.4967]], grad_fn=<MmBackward>)
lkjhgfd
Iter 6/50 - Loss hyperparam: 4.111
kjhgfd
tensor([[0.1570, 0.1574],
        [0.1574, 0.3151]], grad_fn=<MmBackward>)
lkjhgfd
Iter 7/50 - Loss hyperparam: 3.697
kjhgfd
tensor([[0.0866, 0.0869],
        [0.0869, 0.1739]], grad_fn=<MmBackward>)
lkjhgfd
Iter 8/50 - Loss

kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 5/30 - Loss theta2: 298248064.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd


Iter 18/30 - Loss theta2: 129969152.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 19/30 - Loss theta2: 126090896.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03

kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 3/30 - Loss theta2: 1772349696.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd

kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 16/30 - Loss theta2: 269253920.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd

...

tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 1/30 - Loss theta2: 13719409.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([

Iter 14/30 - Loss theta2: 6270169.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 15/30 - Loss theta2: 6096648.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], 

Iter 28/30 - Loss theta2: 5018064.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 29/30 - Loss theta2: 4982612.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], 

Iter 12/30 - Loss theta2: 6471969.500
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
Iter 13/30 - Loss theta2: 6247124.000
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], grad_fn=<MmBackward>)
lkjhgfd
kjhgfd
tensor([[4.4829e-05, 2.2070e-04],
        [2.2070e-04, 1.2577e-03]], 

KeyboardInterrupt: 

In [ ]:
x0 = Tensor([-1.8333])
model.eval()
likelihood.eval()
pr = likelihood((model(x0)))
print(pr.mean)

In [ ]:
model.covar_module(x0,x0).evaluate()

In [ ]:
model.covar_module.forward(x0,x0)

In [ ]:
print((model(x0)).covariance_matrix)